In [ ]:
!pip install autogluon
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularDataset, TabularPredictor

## Setting up Kaggle API Key

In [ ]:
# create the .kaggle directory and an empty kaggle.json file
!mkdir -p ~/.kaggle
!touch ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Fill in your user name and key from creating the kaggle account and API token file
import json
kaggle_username = "FILL_IN_USERNAME"
kaggle_key = "FILL_IN_KEY"

# Save API token the kaggle.json file
with open("/root/.kaggle/kaggle.json", "w") as f:
    f.write(json.dumps({"username": kaggle_username, "key": kaggle_key}))

In [ ]:
# Download the dataset, it will be in a .zip file so you'll need to unzip it as well.
!kaggle competitions download -c titanic
# If you already downloaded it you can use the -o command to overwrite the file
!unzip -o titanic.zip

titanic.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
# check out the data
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_train, df_valid = train_test_split(df_train, test_size=0.2, random_state=42)

# show first rows
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
331,332,0,1,"Partner, Mr. Austen",male,45.5,0,0,113043,28.5000,C124,S
733,734,0,2,"Berriman, Mr. William John",male,23.0,0,0,28425,13.0000,NaN,S
382,383,0,3,"Tikkanen, Mr. Juho",male,32.0,0,0,STON/O 2. 3101293,7.9250,NaN,S
704,705,0,3,"Hansen, Mr. Henrik Juul",male,26.0,1,0,350025,7.8542,NaN,S
813,814,0,3,"Andersson, Miss. Ebba Iris Alfrida",female,6.0,4,2,347082,31.2750,NaN,S


Check the following link to get an understading the data.
https://www.kaggle.com/competitions/titanic/data

# Tasks

1. Perform EDA on the data.
2. Identify the type of task.
3. Identify the type of evaluation metric.
4. Submission. 

# Exploratory Data Analysis

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 331 to 102
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          572 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Cabin        159 non-null    object 
 11  Embarked     710 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 72.3+ KB


In [ ]:
df_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            140
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          553
Embarked         2
dtype: int64

In [ ]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,712.000000,712.000000,712.000000,572.000000,712.000000,712.000000,712.000000
mean,448.234551,0.376404,2.330056,29.498846,0.553371,0.379213,32.586276
std,256.731423,0.484824,0.824584,14.500059,1.176404,0.791669,51.969529
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,224.750000,0.000000,2.000000,21.000000,0.000000,0.000000,7.925000
50%,453.500000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,673.500000,1.000000,3.000000,38.000000,1.000000,0.000000,30.500000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
df_train.Survived.value_counts(normalize=True)

0    0.623596
1    0.376404
Name: Survived, dtype: float64

In [ ]:
df_train.describe(include=object)

,Name,Sex,Ticket,Cabin,Embarked
count,712,712,712,159,710
unique,712,2,558,117,3
top,"Partner, Mr. Austen",male,CA. 2343,C23 C25 C27,S
freq,1,467,7,4,525


In [ ]:
def sex(value):
  if value=='male':
    return 0
  else:
    return 1

def embarked(value):
  if value=='S':
    return 0
  elif value=='C':
    return 1
  else:
    return 2

# df.map()

In [ ]:
def process_data(df):
    df.set_index('PassengerId', inplace=True)
    
    #df['is_Miss'] = df['Name'].str.contains('Miss').astype(int)
    #df['is_Mrs'] = df['Name'].str.contains('Mrs').astype(int)
    #df['is_Master'] = df['Name'].str.contains('Master').astype(int)
    #df['is_Col'] = df['Name'].str.contains('Col').astype(int)
    #df['is_Major'] = df['Name'].str.contains('Major').astype(int)
    #df['is_Mr'] = df['Name'].str.contains('Mr').astype(int)

    drop_columns = ['Name', 'Ticket', 'Cabin']  
    df.drop(drop_columns, axis=1, inplace=True)

    #df['Sex'] = df['Sex'].map(sex)
    #df['Embarked'] = df['Embarked'].map(embarked)
    #df[['Embarked','Sex']] = df[['Embarked', 'Sex']].astype('category')
    df = pd.get_dummies(df)

    df.fillna(0, inplace=True)  
    
    return df

In [ ]:
df_train = process_data(df_train)
df_train.shape

(712, 17)

In [ ]:
df_valid = process_data(df_valid)
df_valid.shape

(179, 17)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 712 entries, 332 to 103
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    712 non-null    int64  
 1   Pclass      712 non-null    int64  
 2   Age         712 non-null    float64
 3   SibSp       712 non-null    int64  
 4   Parch       712 non-null    int64  
 5   Fare        712 non-null    float64
 6   is_Miss     712 non-null    int64  
 7   is_Mrs      712 non-null    int64  
 8   is_Master   712 non-null    int64  
 9   is_Col      712 non-null    int64  
 10  is_Major    712 non-null    int64  
 11  is_Mr       712 non-null    int64  
 12  Sex_female  712 non-null    uint8  
 13  Sex_male    712 non-null    uint8  
 14  Embarked_C  712 non-null    uint8  
 15  Embarked_Q  712 non-null    uint8  
 16  Embarked_S  712 non-null    uint8  
dtypes: float64(2), int64(10), uint8(5)
memory usage: 75.8 KB


In [ ]:
df_train.sample()

,Survived,Pclass,Age,SibSp,Parch,Fare,is_Miss,is_Mrs,is_Master,is_Col,is_Major,is_Mr,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
PassengerId,,,,,,,,,,,,,,,,,
724,0,2,50.0,0,0,13.0,0,0,0,0,0,1,0,1,0,0,1


In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
df_train, df_train['Survived'] = ros.fit_resample(df_train.drop('Survived', axis='columns'), df_train['Survived'])

# Model

In [ ]:
predictor = TabularPredictor(label="Survived").fit(train_data=df_train, time_limit=120, presets="best_quality")

No path specified. Models will be saved in: "AutogluonModels/ag-20221226_185914/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels/ag-20221226_185914/"
AutoGluon Version:  0.6.1
Python Version:     3.8.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    888
Train Data Columns: 16
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generat

In [ ]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                   model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0        LightGBM_BAG_L1   0.884009       0.062240  19.887855                0.062240          19.887855            1       True          4
1    WeightedEnsemble_L2   0.884009       0.065664  20.261350                0.003424           0.373495            2       True          5
2      LightGBMXT_BAG_L1   0.826577       0.088977  39.174144                0.088977          39.174144            1       True          3
3  KNeighborsDist_BAG_L1   0.765766       0.008751   0.005362                0.008751           0.005362            1       True          2
4  KNeighborsUnif_BAG_L1   0.686937       0.008688   0.005221                0.008688           0.005221            1       True          1
Number of models trained: 5
Types of models trained:
{'WeightedEnsembleModel', 'StackerEnsembleMod

{'model_types': {'KNeighborsUnif_BAG_L1': 'StackerEnsembleModel_KNN',
  'KNeighborsDist_BAG_L1': 'StackerEnsembleModel_KNN',
  'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel'},
 'model_performance': {'KNeighborsUnif_BAG_L1': 0.6869369369369369,
  'KNeighborsDist_BAG_L1': 0.7657657657657657,
  'LightGBMXT_BAG_L1': 0.8265765765765766,
  'LightGBM_BAG_L1': 0.884009009009009,
  'WeightedEnsemble_L2': 0.884009009009009},
 'model_best': 'WeightedEnsemble_L2',
 'model_paths': {'KNeighborsUnif_BAG_L1': 'AutogluonModels/ag-20221226_185914/models/KNeighborsUnif_BAG_L1/',
  'KNeighborsDist_BAG_L1': 'AutogluonModels/ag-20221226_185914/models/KNeighborsDist_BAG_L1/',
  'LightGBMXT_BAG_L1': 'AutogluonModels/ag-20221226_185914/models/LightGBMXT_BAG_L1/',
  'LightGBM_BAG_L1': 'AutogluonModels/ag-20221226_185914/models/LightGBM_BAG_L1/',
  'WeightedEnsemble_L2': 'AutogluonModels/ag-20221226_185914/models/

# Evaluation Metric

In [ ]:
performance = predictor.evaluate(df_valid)

Evaluation: accuracy on test data: 0.8324022346368715
Evaluations on test data:
{
    "accuracy": 0.8324022346368715,
    "balanced_accuracy": 0.8312097812097812,
    "mcc": 0.6578633172402021,
    "roc_auc": 0.8984555984555985,
    "f1": 0.8026315789473683,
    "precision": 0.782051282051282,
    "recall": 0.8243243243243243
}


# Complete Model

In [ ]:
df_train = pd.read_csv("train.csv")
df_train = process_data(df_train)

predictor = TabularPredictor(label="Survived").fit(train_data=df_train, time_limit=120, presets="best_quality")

No path specified. Models will be saved in: "AutogluonModels/ag-20221226_190023/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 60s
AutoGluon will save models to "AutogluonModels/ag-20221226_190023/"
AutoGluon Version:  0.6.1
Python Version:     3.8.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Aug 26 08:44:51 UTC 2022
Train Data Rows:    891
Train Data Columns: 16
Label Column: Survived
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generat

In [ ]:
df_test = process_data(df_test)
predictions = predictor.predict(df_test)

# Submission

In [ ]:
submission = pd.read_csv("gender_submission.csv")
submission["Survived"] = predictions.values
submission.to_csv("results.csv", index=False)

In [ ]:
!kaggle competitions submit -c titanic -f results.csv -m "Third model."

100% 2.77k/2.77k [00:02<00:00, 1.01kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster

In [ ]:
!kaggle competitions submissions -c titanic | tail -n +1 | head -n 6

fileName               date                 description             status    publicScore  privateScore  
---------------------  -------------------  ----------------------  --------  -----------  ------------  
results.csv            2022-12-26 19:01:43  Third model.            complete  0.76555                    
results.csv            2022-12-26 18:05:59  Third model.            complete  0.76555                    
results.csv            2022-12-26 17:27:05  Third model.            complete  0.77990                    
results.csv            2022-12-26 17:14:45  Third model.            complete  0.77272                    
